# HW05: Predictive Models

## Setup

Run the following cell if you need to install the PyTorch or Transformers libraries on a lab computer:

In [ ]:
# !pip install diffusers transformers accelerate safetensors torchvision --upgrade
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --upgrade

This code imports the libraries we need to run our inference pipelines:

In [ ]:
from PIL import Image
from transformers import pipeline
from transformers import DetrImageProcessor, DetrForObjectDetection

## Depth [Prediction](https://huggingface.co/depth-anything/Depth-Anything-V2-Base-hf)

In [ ]:
depth_estimator = pipeline(
  task="depth-estimation",
  model="depth-anything/Depth-Anything-V2-Base-hf"
)

In [ ]:
image = Image.open("./imgs/flowers.jpg")
result = depth_estimator(image)
display(result["depth"])

## Object [Detection](https://huggingface.co/facebook/detr-resnet-101)

Some models don't work with the pipeline inference object, but the Transformers library still has some consistent-ish interfaces for running these models.

In [ ]:
OBJ_MODEL = "facebook/detr-resnet-101"
detr_processor = DetrImageProcessor.from_pretrained(OBJ_MODEL)
detr_model = DetrForObjectDetection.from_pretrained(OBJ_MODEL)

In [ ]:
image = Image.open("./imgs/people.jpg")
iw, ih = image.size

detr_inputs = detr_processor(images=image, return_tensors="pt")

detr_output = detr_model(**detr_inputs)
detr_results = detr_processor.post_process_object_detection(detr_output, 0.99, [(ih, iw)])

display(detr_results)

In [ ]:
# Turn label ids into labels
for label_id in detr_results[0]["labels"]:
  print(detr_model.config.id2label[int(label_id)])

## Image to [Text](https://huggingface.co/Salesforce/blip-image-captioning-large)

In [10]:
#code from hugging face to test how it runs 

import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
# >>> a photography of a woman and her dog

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


a photography of a woman and her dog on the beach
a woman sitting on the beach with her dog
